## Portfolio Exercise: Starbucks
<br>

<img src="https://opj.ca/wp-content/uploads/2018/02/New-Starbucks-Logo-1200x969.jpg" width="200" height="200">
<br>
<br>
 
#### Background Information

The dataset you will be provided in this portfolio exercise was originally used as a take-home assignment provided by Starbucks for their job candidates. The data for this exercise consists of about 120,000 data points split in a 2:1 ratio among training and test files. In the experiment simulated by the data, an advertising promotion was tested to see if it would bring more customers to purchase a specific product priced at $10. Since it costs the company 0.15 to send out each promotion, it would be best to limit that promotion only to those that are most receptive to the promotion. Each data point includes one column indicating whether or not an individual was sent a promotion for the product, and one column indicating whether or not that individual eventually purchased that product. Each individual also has seven additional features associated with them, which are provided abstractly as V1-V7.

#### Optimization Strategy

Your task is to use the training data to understand what patterns in V1-V7 to indicate that a promotion should be provided to a user. Specifically, your goal is to maximize the following metrics:

* **Incremental Response Rate (IRR)** 

IRR depicts how many more customers purchased the product with the promotion, as compared to if they didn't receive the promotion. Mathematically, it's the ratio of the number of purchasers in the promotion group to the total number of customers in the purchasers group (_treatment_) minus the ratio of the number of purchasers in the non-promotional group to the total number of customers in the non-promotional group (_control_).

$$ IRR = \frac{purch_{treat}}{cust_{treat}} - \frac{purch_{ctrl}}{cust_{ctrl}} $$


* **Net Incremental Revenue (NIR)**

NIR depicts how much is made (or lost) by sending out the promotion. Mathematically, this is 10 times the total number of purchasers that received the promotion minus 0.15 times the number of promotions sent out, minus 10 times the number of purchasers who were not given the promotion.

$$ NIR = (10\cdot purch_{treat} - 0.15 \cdot cust_{treat}) - 10 \cdot purch_{ctrl}$$

For a full description of what Starbucks provides to candidates see the [instructions available here](https://drive.google.com/open?id=18klca9Sef1Rs6q8DW4l7o349r8B70qXM).

Below you can find the training data provided.  Explore the data and different optimization strategies.

#### How To Test Your Strategy?

When you feel like you have an optimization strategy, complete the `promotion_strategy` function to pass to the `test_results` function.  
From past data, we know there are four possible outomes:

Table of actual promotion vs. predicted promotion customers:  

<table>
<tr><th></th><th colspan = '2'>Actual</th></tr>
<tr><th>Predicted</th><th>Yes</th><th>No</th></tr>
<tr><th>Yes</th><td>I</td><td>II</td></tr>
<tr><th>No</th><td>III</td><td>IV</td></tr>
</table>

The metrics are only being compared for the individuals we predict should obtain the promotion – that is, quadrants I and II.  Since the first set of individuals that receive the promotion (in the training set) receive it randomly, we can expect that quadrants I and II will have approximately equivalent participants.  

Comparing quadrant I to II then gives an idea of how well your promotion strategy will work in the future. 

Get started by reading in the data below.  See how each variable or combination of variables along with a promotion influences the chance of purchasing.  When you feel like you have a strategy for who should receive a promotion, test your strategy against the test dataset used in the final `test_results` function.

In [1]:
# load in packages
from itertools import combinations

from test_results import test_results, score
import numpy as np
import pandas as pd
import scipy as sp
import sklearn as sk

import matplotlib.pyplot as plt
import seaborn as sb
%matplotlib inline

# load in the data
train_data = pd.read_csv('./training.csv')
train_data.head()

,ID,Promotion,purchase,V1,V2,V3,V4,V5,V6,V7
0,1,No,0,2,30.443518,-1.165083,1,1,3,2
1,3,No,0,3,32.159350,-0.645617,2,3,2,2
2,4,No,0,2,30.431659,0.133583,1,1,4,2
3,5,No,0,0,26.588914,-0.212728,2,1,4,2
4,8,Yes,0,3,28.044332,-0.385883,1,1,2,2


In [2]:
promo = train_data[train_data.Promotion=='Yes']
control = train_data[train_data.Promotion=='No']

# Sanity Checks on Invariant Metrics

In [3]:
print("Number of Promotions Sent:",promo.shape[0])
print("Number of Control:",control.shape[0])

Number of Promotions Sent: 42364
Number of Control: 42170


In [4]:
p = .5
alpha = 0.05
p_hat = round(control.shape[0]/train_data.shape[0],4)
sdev = np.sqrt(p*(1-p)/train_data.shape[0])
Z = sp.stats.norm.ppf(1-alpha/2)
CI = np.round([p - Z*sdev, p + Z*sdev],4).tolist()
print("The 95% confidence interval is between:",CI)
print("Our estimate is:",p_hat)
if(p_hat>CI[0] and p_hat<CI[1]):
    print ("Our estimate for p is inside of the 95% confidence interval",
           "so we can proceed with our A/B testing analysis")
else:
    print ("Our estimate for p is outside of the 95% confidence interval",
           "so we should investigate how the data was split more")

The 95% confidence interval is between: [0.4966, 0.5034]
Our estimate is: 0.4989
Our estimate for p is inside of the 95% confidence interval so we can proceed with our A/B testing analysis


# Effect of Promotion

In [5]:
# Conversion Rate of promotion and control groups as well as the overall training data
promo_rate = promo[promo.purchase == 1].shape[0]/promo.shape[0]
print("The conversion rate was",round(promo_rate,4),"for the promotion group")
control_rate = control[control.purchase == 1].shape[0]/control.shape[0]
print("The conversion rate was",round(control_rate,4),"for the control group")
pooled_rate = train_data[train_data.purchase == 1].shape[0]/train_data.shape[0]
print("The conversion rate was",round(pooled_rate,4),"for the entire dataset")

The conversion rate was 0.017 for the promotion group
The conversion rate was 0.0076 for the control group
The conversion rate was 0.0123 for the entire dataset


In [6]:
sd_pooled = np.sqrt(pooled_rate*(1-pooled_rate)*(1/control.shape[0]+1/promo.shape[0]))
Z = sp.stats.norm.ppf(1-alpha/2)
ME = round(Z*sd_pooled,4)
diff = round(promo_rate-control_rate,4)
CI = np.round([(diff - ME)*100, (diff + ME)*100],4).tolist()
print("The conversion rate rose from",round(control_rate,4),"% to",round(promo_rate,4),"%")
print("The Incremental Response Rate due to the experiment was about",diff*100,"%")
print("The 95% confidence interval for the Incremental Response Rate is:",CI,"%")

The conversion rate rose from 0.0076 % to 0.017 %
The Incremental Response Rate due to the experiment was about 0.95 %
The 95% confidence interval for the Incremental Response Rate is: [0.8, 1.1] %


Since our 95% confidence interval excludes 0 we can say that this promotion was statistically significant. Now we need to see if it was practically significant, meaning does it make sense to financially implement.

In [7]:
NIR_mid = (10*promo_rate-.15)-10*control_rate
NIR_lower = (10*(promo_rate - ME)-.15)-10*control_rate
NIR_upper = (10*(promo_rate + ME)-.15)-10*control_rate
print("Based on the data from this experiment the net incremental revenue per person is",
      round(NIR_mid,4))
print("We expect the Net Incremental Revenue to be between",round(NIR_lower,4),"and",round(NIR_upper,4))

Based on the data from this experiment the net incremental revenue per person is -0.0555
We expect the Net Incremental Revenue to be between -0.0705 and -0.0405


Because the Net Incremental Revenue from the experiment is negative we would not advise rolling this experiment out to all starbucks customers. There was an increase in the purchase rate but that increase was more than offset by the costs of sending out this promotion. If we could narrow down the list of users to send this promotion to it may merit further investigation though.

# Optimizing who should recieve the promotion

In [8]:
promo.head()

,ID,Promotion,purchase,V1,V2,V3,V4,V5,V6,V7
4,8,Yes,0,3,28.044332,-0.385883,1,1,2,2
8,15,Yes,0,2,31.930423,0.393317,2,3,1,2
10,17,Yes,0,1,32.770916,-1.511395,2,1,4,1
12,20,Yes,0,1,36.957009,0.133583,2,3,1,1
14,22,Yes,0,3,36.911714,-0.905350,2,2,4,1


First we need to select our target and predictor values

In [21]:
X = promo.loc[:,'V1':'V7']
y = promo.purchase.values

Then we should split the data into training and validation notice since the data is unbalanced we should stratify on y to keep the same percentage of purchases in test and training

In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1842, stratify = y)

now lets try to fit an XGBoost model to the data to predict y from X

In [23]:
!pip install xgboost

In [24]:
from xgboost import XGBClassifier
model = XGBClassifier()
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1)

Now lets use the trained model to predict for values in the test set

In [25]:
y_pred = model.predict(X_test)
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

             precision    recall  f1-score   support

          0       0.98      1.00      0.99      8329
          1       0.00      0.00      0.00       144

avg / total       0.97      0.98      0.97      8473



/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Ouch! this data is really unbalanced which leaves our model predicting that nobody in the dataset should recieve an offer! To handle this we either need to scale the weights, oversample the minority class or undersample the majority class.

In [26]:
scale = np.sum(y_train==0)/np.sum(y_train==1)
print("we should scale the positive weight by",scale)

we should scale the positive weight by 57.7365684575


In [27]:
model_scaled_weights = XGBClassifier(scale_pos_weight = scale)
model_scaled_weights.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=57.736568457538993, seed=None,
       silent=None, subsample=1, verbosity=1)

In [28]:
y_pred = model_scaled_weights.predict(X_test)
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

             precision    recall  f1-score   support

          0       0.99      0.58      0.73      8329
          1       0.03      0.64      0.05       144

avg / total       0.97      0.58      0.72      8473



Practice turning 0 and 1 into Yes and No

In [29]:
pd.Series(y_pred).map({True:"Yes", False:"No"}).values

array(['Yes', 'Yes', 'Yes', ..., 'Yes', 'Yes', 'No'], dtype=object)

Just by scaling the weights we can get a 64% recall meaning we captured around 64% of the total number of people in the dataset that would have made a purchase

In [30]:
model_scaled_weights_full_training = XGBClassifier(scale_pos_weight = scale)
model_scaled_weights.fit(X, y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=57.736568457538993, seed=None,
       silent=None, subsample=1, verbosity=1)

In [31]:
def promotion_strategy(df):
    '''
    INPUT 
    df - a dataframe with *only* the columns V1 - V7 (same as train_data)

    OUTPUT
    promotion_df - np.array with the values
                   'Yes' or 'No' related to whether or not an 
                   individual should recieve a promotion 
                   should be the length of df.shape[0]
                
    Ex:
    INPUT: df
    
    V1	V2	  V3	V4	V5	V6	V7
    2	30	-1.1	1	1	3	2
    3	32	-0.6	2	3	2	2
    2	30	0.13	1	1	4	2
    
    OUTPUT: promotion
    
    array(['Yes', 'Yes', 'No'])
    indicating the first two users would recieve the promotion and 
    the last should not.
    '''
    y_pred = model_scaled_weights.predict(df)
    
    promotion = pd.Series(y_pred).map({True:"Yes", False:"No"}).values
    
    return promotion

In [32]:
# This will test your results, and provide you back some information 
# on how well your promotion_strategy will work in practice

test_results(promotion_strategy)

Nice job!  See how well your strategy worked on our test data below!

Your irr with this strategy is 0.0192.

Your nir with this strategy is 336.80.
We came up with a model with an irr of 0.0188 and an nir of 189.45 on the test set.

 How did you do?


(0.019247585510520599, 336.79999999999995)